In [15]:
import os
import sys
import pandas as pd

sys.path.append('../pytorch_model_helpers')
from pytorch_helpers import drug_stratification

import umap
from sklearn.decomposition import PCA,FactorAnalysis
from sklearn.preprocessing import StandardScaler,QuantileTransformer
from sklearn.metrics import precision_recall_curve,log_loss
from sklearn.metrics import average_precision_score,roc_auc_score
from sklearn.feature_selection import VarianceThreshold
import os
import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [4]:
data_dir = "../../2.data_split/model_data/L1/"

In [5]:
df_train = pd.read_csv(os.path.join(data_dir, 'train_shuffle_lvl4_data.csv.gz'),
                                   compression='gzip',low_memory = False)

In [6]:
df_targets = pd.read_csv(os.path.join(data_dir, 'target_labels.csv'))

In [ ]:
metadata_cols = ['Metadata_broad_sample', 'pert_id', 'pert_idose', 'replicate_id', 
                         'pert_iname', 'moa', 'sig_id', 'det_plate', 'dose', 'det_well']
target_cols = df_targets.columns[1:]
NFOLDS = 5

In [30]:
df = df_train.copy()
nfold=NFOLDS
col_name="replicate_id"
cpd_freq_num=24

In [31]:
drug_value_ct = df['pert_iname'].value_counts()
drug_vc1 = drug_value_ct.loc[drug_value_ct <= cpd_freq_num].index.sort_values()
drug_vc2 = drug_value_ct.loc[drug_value_ct > cpd_freq_num].index.sort_values()
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=nfold, shuffle=True, random_state=33)
df_drug_vc1 = df.groupby('pert_iname')[target_cols].mean().loc[drug_vc1]

/home/ubuntu/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=33 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [32]:
# STRATIFY DRUGS X OR LESS based on each specific drug/compound
for fold,(idxT,idxV) in enumerate(skf.split(df_drug_vc1,df_drug_vc1[target_cols])):
    drugs_fold = {drugs:fold for drugs in df_drug_vc1.index[idxV].values}
    dct1.update(drugs_fold)

In [33]:
# STRATIFY DRUGS X OR MORE based on the drug's replicates
skf = MultilabelStratifiedKFold(n_splits=nfold, shuffle=True)

/home/ubuntu/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [34]:
df_drug_vc2 = df.loc[df.pert_iname.isin(drug_vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate(skf.split(df_drug_vc2,df_drug_vc2[target_cols])):
    drugs_fold = {drugs:fold for drugs in df_drug_vc2[col_name][idxV].values}
    dct2.update(drugs_fold)

In [35]:
##fold column
df['fold'] = df.pert_iname.map(dct1)
df.loc[df.fold.isna(),'fold'] = df.loc[df.fold.isna(),col_name].map(dct2)
df['fold'] = df['fold'].astype(int)

In [12]:
drug_stratification(
    df_train,
    NFOLDS,
    target_cols,
    col_name='replicate_id',
    cpd_freq_num=36
)

/home/ubuntu/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=33 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/ubuntu/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: Found array with 0 sample(s) (shape=(0, 501)) while a minimum of 1 is required.